In [ ]:
#Generate tuples (Question, Token, SemType) - old approach + old code - don't run this cell now
%cd ~/Desktop/CDQA-project

from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import re
s = pd.read_json('text.out')

documents = s.iloc[0][0]['Document']['Utterances']
Metamap_Tokenizations = [] #To be used for final QA

def retrieve_tokens(SyntaxUnits):
    tokens = []
    for i in range(len(SyntaxUnits)):
        tokens.append(SyntaxUnits[i]['InputMatch'])
    return tokens

def retrieve_mappings(Mappings):
    mapped_semantic_types = []
    #No mappings found
    if len(Mappings) == 0:
        return [] #These words will get their embeddings from BERT
    else:
        candidates = Mappings[0]['MappingCandidates'] #Choosing Only top mappings
        for cnd in candidates:
            mapped_semantic_types.append([' '.join(cnd['MatchedWords']), cnd['CandidateCUI'], \
                                              cnd['CandidatePreferred'], cnd['SemTypes'][0]])
            entities.add(cnd['CandidatePreferred'])
    return mapped_semantic_types

entities = set()
for doc in tqdm(documents):
    Phrases = doc['Phrases']
    Phrase_Tokenizations = []
    Mappings = []
    for ph in Phrases:
        Phrase_Tokenizations.append(retrieve_tokens(ph['SyntaxUnits']))
        Mappings.append(retrieve_mappings(ph['Mappings']))
    #Flattening the Lists
    Phrase_Tokenizations = [item for sublist in Phrase_Tokenizations for item in sublist]
    Mappings = [item for sublist in Mappings for item in sublist]
    #Creating the final list
    Metamap_Tokenizations.append((doc['UttText'], Phrase_Tokenizations, Mappings))
    
#Removing extra spaces from each question
for index, tup in enumerate(Metamap_Tokenizations):
    temp = list(tup)
    #The only question which was causing an issue
    if type(temp[0]) != str:
        temp[0] = 'What were the impact of event scale–revised scores?'
    else:
        temp[0] = temp[0].strip()
    Metamap_Tokenizations[index] = tuple(temp)
    
print(f"Number of entities discovered: {len(entities)}")

#Replacing each shorthand mapping with KG concept - IF USING SN
import mysql.connector

mydb = mysql.connector.connect(host="localhost", user="root", password="Saptarshi1234!", database="umls")
mycursor = mydb.cursor()

for i in range(len(Metamap_Tokenizations)):
    for j in range(len(Metamap_Tokenizations[i][2])):
        mycursor.execute("select STY_RL from SRDEF where ABR = '%s' " % Metamap_Tokenizations[i][2][j][3])
        Metamap_Tokenizations[i][2][j][3] = mycursor.fetchall()[0][0]

mycursor.close()

#Saving Metamap_Tokenizations for use during question embedding creation
pd.DataFrame(Metamap_Tokenizations, columns=['Question','Tokenization',\
                                            'Mappings']).to_pickle('Metamap_Tokenizations.pkl')

'''--->Extracting relations from the Semantic Network<---'''
    #Generating possible semantic types connected to the current one
    mycursor.execute('''SELECT STY_RL2, RL
                        FROM SRSTR 
                        WHERE STY_RL1 = '%s';''' % row.Semantic_Type)
    possible_semantic_types_2 = [STREL(*x) for x in mycursor.fetchall()]
    for sem_type_2 in possible_semantic_types_2:
        if sem_type_2.ST == None:
            continue
        ST2 = sem_type_2.ST.decode()
        REL = sem_type_2.REL.decode()
        for result in CUI_Preferred_Concept_Semantic_Type_Lookup_Table.query('Semantic_Type==@ST2').itertuples(index=False):
            KGT.add((row.Preferred_Concept, REL, result.Preferred_Concept))      
    

In [ ]:
#Just need to run this cell once to update the dataset with the cleaned questions - done
import json
import os
data_path = os.path.relpath('data/COVID-QA-original.json')

dataset = json.load(open(data_path, 'r'))
with open('Cleaned_Questions_utf8_replaced.txt','r') as file:
    cleaned_questions = file.read().split('\n')
    
index = 0
for i in range(len(dataset['data'])):
    for question_dictionary in dataset['data'][i]['paragraphs'][0]['qas']:
        question_dictionary['question'] = cleaned_questions[index]
        index += 1
        
with open('COVID-QA_cleaned.json', 'w') as f:
    json.dump(dataset, f, ensure_ascii=False, indent=3)

In [1]:
#Generate tuples (question, (matched_text, CUI, preferred candidate)) - new approach
import json
import pandas as pd

texts = json.load(open('MM_NEW.json'))['AllDocuments'][0]['Document']['Utterances']

Metamap_Tokenizations = []
for ques in texts:
    mappings = []
    question_text = ques['UttText']
    ques_start_idx = int(ques['UttStartPos'])
    for phr in ques['Phrases']:
        if phr['Mappings'] != []:
            for phr_dict in phr["Mappings"][0]['MappingCandidates']: #Choosing the first candidate
                start_idx = int(phr_dict['ConceptPIs'][0]['StartPos']) - ques_start_idx
                end_idx = start_idx + int(phr_dict['ConceptPIs'][0]['Length'])
                mappings.append((question_text[start_idx:end_idx], phr_dict['CandidateCUI'], \
                                 phr_dict['CandidatePreferred']))
    Metamap_Tokenizations.append((question_text, mappings))

entities = set()
for mappings in Metamap_Tokenizations:
    for tup in mappings[1]:
        entities.add(tup[2])
print(f"Number of entities discovered: {len(entities)}")

#Saving Metamap_Tokenizations for use during question embedding creation
pd.DataFrame(Metamap_Tokenizations, columns=['Question', 'Mappings']).to_pickle('Metamap_Tokenizations.pkl')

Number of entities discovered: 1896


In [2]:
#New version of CUI-PC_Lookup table
cuis = [y[1] for x in Metamap_Tokenizations for y in x[1]]
pc = [y[2] for x in Metamap_Tokenizations for y in x[1]]
CUI_Preferred_Concept_Lookup_Table = pd.DataFrame(zip(cuis, pc), columns=['CUI','Preferred_Concept']).drop_duplicates()

In [3]:
#New version of creating KGT (only Metathesaurus)
import sqlite3
import mysql.connector
from tqdm import tqdm
from collections import namedtuple

mydb = mysql.connector.connect(host="localhost", user="root", password="Saptarshi1234!", database="umls")
mydb.set_charset_collation('latin1')
mycursor = mydb.cursor()

#For sqlite (Metathesaurus)
conn = sqlite3.connect('umls.db')
cursor = conn.cursor()

#All Knowledge Graph Triples
KGT = set()

#To make the code more readable (pythonic)
CUIREL = namedtuple('CUIREL', ['CUI', 'REL'])
#STREL = namedtuple('STREL', ['ST', 'REL'])

for row in tqdm(CUI_Preferred_Concept_Lookup_Table.itertuples(index=False)):
    '''--->Extracting relations from the Metathesaurus<---'''
    #This gives the "incoming" relations i.e. CUI2 - RELA - CUI1
    incoming_relations = []
    cursor.execute('''SELECT DISTINCT CUI2, RELA
                                  FROM MRREL
                                  WHERE CUI1 = '%s' AND RELA <> '';''' % row.CUI)
    results = cursor.fetchall()
    #Checking which of the returned tuples are present in our lookup table 
    for tupA in results:
        CUIREL2 = CUIREL(*tupA)
        for tupB in CUI_Preferred_Concept_Lookup_Table.query('CUI==@CUIREL2.CUI').itertuples(index=False):
            KGT.add((tupB.Preferred_Concept, CUIREL2.REL, row.Preferred_Concept))

    #This gives the "outgoing" relations i.e. CUI1 - RELA - CUI2
    outgoing_relations = []
    cursor.execute('''SELECT DISTINCT CUI1, RELA 
                                  FROM MRREL
                                  WHERE CUI2 = '%s' AND STYPE2 = 'CUI' AND RELA <> '';''' % row.CUI)
    results = cursor.fetchall()
    #Checking which of the returned tuples are present in our lookup table
    for tupA in results:
        CUIREL2 = CUIREL(*tupA)
        for tupB in CUI_Preferred_Concept_Lookup_Table.query('CUI==@CUIREL2.CUI').itertuples(index=False):
            KGT.add((row.Preferred_Concept, CUIREL2.REL, tupB.Preferred_Concept))
    
    
conn.close()
mycursor.close()

print(f"Number of triples extracted: {len(KGT)}")

1896it [21:22,  1.48it/s]

Number of triples extracted: 6808


In [4]:
#Creating Train/Validation/Test splits for training KGE's
import pandas as pd
import numpy as np
import os

#Converting set to pandas dataframe for easy splitting
KGT = pd.DataFrame(KGT)

#Giving the KGT dataframe meaningful column names
KGT.rename(columns={0: "E1", 1: "Rel", 2: "E2"}, inplace=True)

#80/10/10 split
train, validation, test = np.split(KGT.sample(frac=1, random_state=42), [int(.8*len(KGT)), int(.9*len(KGT))])

source_of_KG = 'MT'

#Creating folder where dataset files will be saved
try:
    os.mkdir(os.path.join(os.path.abspath(os.getcwd()), f"UMLS_KG_{source_of_KG}"))
    print("KG Directory Created")
except:
    print("KG Directory Already Exists")

dataset_path = os.path.abspath(f"UMLS_KG_{source_of_KG}")

#Saving datasets as .txt files to be used for training the KGE
np.savetxt(os.path.join(dataset_path, f'UMLS_KG_{source_of_KG}-train.txt'), train.values, delimiter="\t", fmt="%s")
np.savetxt(os.path.join(dataset_path, f'UMLS_KG_{source_of_KG}-valid.txt'), validation.values, delimiter="\t", fmt="%s")
np.savetxt(os.path.join(dataset_path, f'UMLS_KG_{source_of_KG}-test.txt'), test.values, delimiter="\t", fmt="%s")

print('KG dataset saved...')

KGT.to_pickle(os.path.join(dataset_path, "KGT.pkl"))
print('KGT saved as dataframe')

KG Directory Created
KG dataset saved...
KGT saved as dataframe
